In [1]:
# 오라클DB 접속
import oml
import oracledb
oml.core.methods.__embed__ = False

connection = oracledb.connect(user="vector", password="vector", dsn="localhost:1521/freepdb1")
cursor = connection.cursor()

# 버전 확인
dbversion = """select product,VERSION_FULL from product_component_version"""
result=cursor.execute(dbversion).fetchall()
print('DB version : ',result)

DB version :  [('Oracle Database 23ai Free', '23.7.0.25.01')]


In [2]:
# blob 테이블 쿼리

tblq = """select * from laws_doc"""
cursor.execute(tblq)
row = cursor.fetchall()
display(row)

[(1, '국군조직법법률제10821호_20111015', <oracledb.LOB at 0x7f1ea928ee70>, None),
 (2, '국방대학교설치법법률제14609호_20170622', <oracledb.LOB at 0x7f1eae6b5b20>, None),
 (3, '군수품관리법법률제17997호_20210413', <oracledb.LOB at 0x7f1eae6e35f0>, None),
 (4, '군형법법률제18465호_20220701', <oracledb.LOB at 0x7f1ea948f860>, None),
 (5, '예비군법법률제19082호_20230614', <oracledb.LOB at 0x7f1ea9687920>, None),
 (6,
  '군용항공기운용등에관한법률_법률제19573호_20240126',
  None),
 (7, '군사법원법법률제19839호_20240118', <oracledb.LOB at 0x7f1f22e0ecf0>, None),
 (8, '군인공제회법법률제19949호_20240710', <oracledb.LOB at 0x7f1f22e0e9f0>, None),
 (9,
  '국방군사시설사업에관한법률법률제20010호_20240717',
  None),
 (10,
  '군인의지및복무에관한 기본법_법률_제20189호_20240807',
  None),
 (11, '군인보수법법률제20637호_20250107', <oracledb.LOB at 0x7f1f22e0e450>, None),
 (12, '방위사업법법률제20644호_20250107', <oracledb.LOB at 0x7f1f22e0c1a0>, None),
 (13, '군무원인사법법률제20799호_20250919', <oracledb.LOB at 0x7f1f22e0c830>, None),
 (14, '방어해면법법률제20806호_20250318', <oracledb.LOB at 0x7f1f22e0c980>, None),
 (15,
  '제2연평해전전사자보상에관한특별법_법률제19

In [10]:
# hwpx 워드에서 텍스트 추출
# 사용 유틸리티 : utl_to_text 
DocId = input("문서번호를 입력해주세요(예:19)")
export_q = """
SELECT dbms_vector_chain.utl_to_text(dt.laws,json('{
                                      "plaintext": "true",
                                      "charset"  : "UTF8" 
           }')) as text 
from laws_doc dt 
where docid = :docid"""

cursor.execute(export_q,docid=DocId)
row = cursor.fetchall()
outputs = row[0][0].read()
#print(outputs)

import re
str = re.sub("[\n]", "", outputs)
print(str)

문서번호를 입력해주세요(예:19) 1


/opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60001 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60002 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60003 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60004 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60005 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60006 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60007 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60008 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60009 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60010 /opt/oracle/product/23ai/dbhomeFree/ctx/tmp/drgityCQIQ60011 application/hwp+zipko 2025-03-27T01:29:38Z2025-03-27T01:29:38Z2025년 2월 27일 목요일 오전 10:29:38 ^1.^2.^3)^4) 국군조직법 [시행 2011. 10. 15.] [법률 제10821호, 2011. 7. 14., 일부개정] 제1장 총칙 제1조(목적) 제2조(국군의 조직) 제3조(각군의 주임무 등) 제4조(군인의 신분 등) 제5조(군기) 제2장 군사권한 제6조(대통령의 지위와 권한) 제7조 제8조(국방부장관의 권한) 제9조(합동참모의장의 권한) 제10조(각군 참모총장의 권한 등) 제11조(소속 부서의 장의 권한) 제3장 합동참모본부 제12

In [4]:
# hwpx 파일에서 텍스트 추출, 100개 단어 단위로 청킹하는 쿼리
# 사용 유틸리티 : utl_to_text, utl_to_chunks
# 청킹 쿼리
print("문서 "+ DocId + " 청킹합니다.")

chunk_q = """
SELECT S.docid, 
    JSON_VALUE(C.column_value, '$.chunk_id' RETURNING NUMBER) AS chunk_id,
    JSON_VALUE(C.column_value, '$.chunk_offset' RETURNING NUMBER) AS chunk_pos,
    JSON_VALUE(C.column_value, '$.chunk_length' RETURNING NUMBER) AS chunk_siz,
    JSON_VALUE(C.column_value, '$.chunk_data') AS txt
from laws_doc S, dbms_vector_chain.utl_to_chunks(
            dbms_vector_chain.utl_to_text(
                s.laws, json('{"plaintext":"true","charset":"UTF8"}')),
            JSON('{ "by"       : "words",
                    "max"      : "100",
                    "overlap"  : "0",
                    "split"    : "newline",
                    "language" : "korean",
                    "normalize": "all" }')
) C
where S.docid = :docid
"""
# 출력 포맷 조절 및 불필요 문자 제거
import pandas as pd

cursor.execute(chunk_q,docid=DocId)
row = cursor.fetchall()

# 회면 출력

tbl = pd.DataFrame(row)

colnm = cursor.description
col=[]

for i in colnm:
    col.append(i[0])

tbl.columns = col

tbl['TXT'] = tbl['TXT'].str.replace("[\n]","", regex=False)
#print()

pd.set_option('display.max_colwidth', 200)

display(tbl)

문서 19 청킹합니다.


,DOCID,CHUNK_ID,CHUNK_POS,CHUNK_SIZ,TXT
0,19,1,13,368,공공데이터 품질관리 수준평가 가이드 \nv 1.0\n\n2016. 6\n\n목 차 \nⅠ. 공공데이터 품질관리 수준 평가 개요 \n1. 평가 목적 4\n\n2. 추진 근거 4\n\n3. 평가 대상 5\n\n4. 평가 기준 5\n\n5. 추진 체계 8\n\n6. 평가 절차 9\n\n7. 평가 방식 10\n\n8. 평가 결과 판정 11\n\nⅡ. 품질관...
1,19,2,381,457,4. 데이터 표준관리 21\n\n5. 데이터 구조관리 27\n\n6. 연계데이터 관리 32\n\n【평가영역】운영단계 품질관리\n\n7. 데이터 품질 진단 37\n\n8. 데이터 정제 43\n\n9. 근본원인 분석 및 해결 47\n\n【평가영역】활용단계 품질관리\n\n10. 데이터 개방관리 51\n\n11. 데이터 품질 성과관리 58\n\nⅢ. 개방데...
2,19,3,839,252,「공공데이터의 제공 및 이용 활성화에 관한 법률」(이하「공공데이터법」이라 함) 제22조(공공데이터의 품질관리) 제3항\n\n☞ 제22조(공공데이터의 품질관리) ③ 행정자치부장관은 미래창조과학부장관과 협의하여 제2항에 따라 정기적으로 사회적·경제적 파급효과가 큰 제공대상 공공데이터에 대한 품질 진단·평가를 실시하고 그 결과를 공표할 수 있다. \n「...
3,19,4,1091,351,☞ 제17조(공공데이터의 품질 진단 및 개선) ① 법 제22조제3항에 따른 공공데이터의 품질 진단ㆍ평가의 기준은 다음 각 호와 같다. \n1. 공공데이터 구조 및 성능\n\n2. 공공데이터 품질관리체계\n\n3. 공공데이터 표준화 준수\n\n4. 공공데이터 값 오류\n\n5. 그 밖에 행정자치부장관이 필요하다고 인정하는 사항\n\n② 행정자치부장관은...
4,19,5,1442,327,3. 품질 진단ㆍ평가의 절차 및 기간\n\n4. 품질 진단ㆍ평가 결과에 따른 시정조치\n\n5. 그 밖에 품질 진단ㆍ평가 시행에 필요한 사항\n\n③ 행정자치부장관은 법 제22조제3항에 따라 공공데이터의 품질 진단ㆍ평가를 실시하려는 경우에는 진단ㆍ평가의 대상이 되는 공공데이터를 보유하고 있는 공공기관의 장에게 제2항에 따른 품질 진단ㆍ평가 계획을 미...
...,...,...,...,...,...
142,19,143,49179,343,"(예시) 회원 테이블에 존재하는 회원만 교육신청이 가능한데, 회원테이블에 존재하지 않는 회원이 교육신청자로 등록된 경우\n\n[계산식] 오류율(%) = (진단대상 컬럼의 전체 오류 데이터 건수 / 진단대상 컬럼의 전체 데이터 건수) × 100\n\n- 번호 정확성\n\n[진단내용] 정해진 규칙 등에 따라 관리되는'번호'도메인의 데이터를 관리하는 컬럼..."
143,19,144,49522,321,[계산식] 오류율(%) = (진단대상 컬럼의 전체 오류 데이터 건수 / 진단대상 컬럼의 전체 데이터 건수) × 100\n\n- 시간순서 일관성\n\n[진단내용] 날짜 도메인 컬럼 중에서 날짜의 선후관계가 명확히 정의된 컬럼의 데이터 오류 측정 \n[진단기준] 날짜 도메인 컬럼 중에서 날짜의 선후관계를 위배한 데이터 추출\n\n(예시) 시작일시와 종료...
144,19,145,49843,289,"[진단내용] 컬럼 간 관계에 따른 특정 컬럼의 논리적 일관성 오류를 측정 \n[진단기준] 컬럼 간 관계에 따른 특정 컬럼의 논리적 일관성을 위배하는 데이터 추출\n\n(예시) 가입여부, 가입일시 : 가입여부가 가입(""Y"")인 경우, 가입일시는 반드시 값이 존재해야 함\n\n(예시) 혼인여부, 결혼기념일 : 혼인여부가 기혼(""Y"")인 경우, 결혼기념일..."
145,19,146,50132,320,"- 계산 및 집계 간 일관성\n\n[진단내용] 원천데이터의 계산 등을 통해 저장되는 컬럼 값이 정확하게 관리되고 있는지를 측정 \n[진단기준] 계산 값을 저장하는 컬럼 값과 원천데이터의 계산 값이 상이한 데이터를 추출\n\n(예시) 구매건수, 구매금액, 구매합계 : 구매건수×구매금액 값이 구매합계와 일치해야 함\n\n[계산식] 오류율(%) = (진단..."


In [5]:
# 유즈케이스를 위한 샘플 테이블 생성
create_table_v_sql = """create table if not exists doc_vector(doc_id number,embed_id number,embed_data varchar2(4000),embed_vector vector)"""
cursor.execute(create_table_v_sql)
print("complete")

complete


In [6]:
# 체인어블 임베딩 데모

# 테이블에 저장된 hwp 파일을 불러서 텍스트 추출, 청킹 후 임베딩해서 별도 테이블(doc_vector)에 저장
# 사용 유틸리티 : ult_to_text, utl_to_chunks, utl_to_embedding
# 임베딩 모델 : all_minilm_l6_v2.onnx(doc_model_han으로 DB에서 관리)
# 체인어블 임베딩 쿼리
init_vector = """delete from doc_vector where doc_id = :docid """
cursor.execute(init_vector,docid=DocId)
connection.commit()
#print("문서 "+ DocId + " 청킹이 삭제 됐습니다")
#print("문서 데이터 초기화 완료")

# 싱글 파이프라인 임베딩
print("문서 "+ DocId + " 청킹, 임베딩합니다.")
gen_data = """
 insert into doc_vector
    select dt.docid, et.embed_id, et.embed_data, to_vector(et.embed_vector) embed_vector
    from laws_doc dt,
         dbms_vector.utl_to_embeddings(
             dbms_vector_chain.utl_to_chunks(
                    dbms_vector_chain.utl_to_text(
                         dt.laws, JSON('{"plaintext":"true","charset":"UTF8"}')),
             JSON('{ "by"       : "words",
                     "max"      : "100",
                    "overlap"  : "0",
                    "split"    : "newline",
                    "language" : "korean",
                    "normalize": "all" }')),
        json('{"provider":"database", "model":"multilingual_e5_small"}')) t,
    json_table(t.column_value, '$' columns (embed_id number,embed_data varchar2(4000), embed_vector CLOB)) et
    where dt.docid = :docid """

cursor.execute(gen_data,docid=DocId)
connection.commit()
print("문서 "+ DocId + " 청킹이 완료됐습니다")

문서 19 청킹, 임베딩합니다.
문서 19 청킹이 완료됐습니다


In [7]:
# 임베딩 결과 조회
import pandas as pd
print("문서 "+ DocId + " 임베딩 결과")
gendata_q = """select * from doc_vector where doc_id = :docid"""
cursor.execute(gendata_q,docid=DocId)
row = cursor.fetchall()

colnm = cursor.description
col=[]

for i in colnm:
    col.append(i[0])

tbl = pd.DataFrame(row)

tbl.columns = col

tbl['EMBED_DATA'] = tbl['EMBED_DATA'].str.replace("\n","", regex=False)
#print()

pd.set_option('display.max_colwidth', 50)

display(tbl)


문서 19 임베딩 결과


,DOC_ID,EMBED_ID,EMBED_DATA,EMBED_VECTOR
0,19,132,2등급  개방데이터의 품질 점검결과 오류율이 0.5% 미만이어야 한다. 1등급 ...,"[0.021052422001957893, -0.059270452708005905, ..."
1,19,133,"※ 평가대상DB가 2구간 이상인 경우, 각 구간 별 기준에 따라 무작위로 대상을 선...","[0.049743879586458206, -0.03647041320800781, 0..."
2,19,134,[진단기준] 대상DB 전체 테이블의 분류 도메인 컬럼에서 기본 값 또는 유효값을 제...,"[0.030544474720954895, 0.002571820979937911, -..."
3,19,38,지표설명  데이터 표준코드를 정의한다. - 표준코드 정의 시 필요한 생성규칙을 정...,"[0.02578602358698845, -0.015547703951597214, -..."
4,19,39,표준코드 생성규칙을 정의하여야 한다. 표준코드는 표준코드 생성규칙을 준수하여야 한...,"[0.04566666856408119, 0.006407597102224827, -0..."
...,...,...,...,...
142,19,83,3등급  4등급을 충족하여야 한다.  DB에 저장된 데이터의 관계 정합성 검증을...,"[0.046362511813640594, -0.02392665110528469, -..."
143,19,84,데이터 품질진단 현황을 계량화(지표)하여 관리하여야 한다. ※ 예시) 품질진단 건수...,"[0.039130501449108124, -0.03535730019211769, -..."
144,19,129,2등급  품질관리 성과측정을 위한 지표를 정의하여야 한다.  정의된 지표를 활용...,"[0.02465311996638775, -0.04645797610282898, -0..."
145,19,130,※ 만족도 조사는 평가기관에서 일괄적으로 조사 후 100점 만점 기준으로 점수화 한...,"[0.06581097096204758, -0.038356006145477295, -..."


In [8]:
# drop index
cursor.execute("drop index doc_vector_idx")
print("droped index")
# 벡터 인덱스 생성
cursor.execute("create vector index doc_vector_idx on doc_vector(embed_vector) ORGANIZATION INMEMORY NEIGHBOR GRAPH WITH DISTANCE COSINE")
print("Created vector index")

droped index
Created vector index


In [9]:
# 벡터 검색 

user_q = input("질문을 입력해주세요? : ") # 공공데이터 품질관리 절차 

sim_q = """select doc_id, embed_id, embed_data, vector_distance(embed_vector, vector_embedding(MULTILINGUAL_E5_SMALL using :user_q as data), COSINE) Distance
from doc_vector 
order by vector_distance(embed_vector, vector_embedding(MULTILINGUAL_E5_SMALL using :user_q as data), COSINE)
fetch first 5 rows only 
"""

cursor.execute(sim_q, user_q = user_q)
row = cursor.fetchall()

tbl = pd.DataFrame(row)

colnm = cursor.description
col=[]

for i in colnm:
    col.append(i[0])

tbl.columns = col

tbl['EMBED_DATA'] = tbl['EMBED_DATA'].str.replace("\n","", regex=False)
#print()

pd.set_option('display.max_colwidth', None)
display(tbl)

질문을 입력해주세요? :  공공데이터 품질관리 절차 


,DOC_ID,EMBED_ID,EMBED_DATA,DISTANCE
0,19,2,"4. 데이터 표준관리 215. 데이터 구조관리 276. 연계데이터 관리 32【평가영역】운영단계 품질관리7. 데이터 품질 진단 378. 데이터 정제 439. 근본원인 분석 및 해결 47【평가영역】활용단계 품질관리10. 데이터 개방관리 5111. 데이터 품질 성과관리 58Ⅲ. 개방데이터 품질 평가지표 61Ⅰ. 공공데이터 품질관리 수준 평가 개요 1. 평가 목적 공공데이터의 품질관리 수준을 평가하여 안정적이고 지속적인 공공데이터의 품질관리 수준 향상을 유도하고, 이를 통해 지속적·안정적인 고품질 공공데이터의 개방, 활용에 기여하고자 함2. 추진근거",0.065892
1,19,1,공공데이터 품질관리 수준평가 가이드 v 1.02016. 6목 차 Ⅰ. 공공데이터 품질관리 수준 평가 개요 1. 평가 목적 42. 추진 근거 43. 평가 대상 54. 평가 기준 55. 추진 체계 86. 평가 절차 97. 평가 방식 108. 평가 결과 판정 11Ⅱ. 품질관리체계 평가지표【평가영역】계획단계 품질관리1. 데이터 품질 정책 수립 142. 데이터 품질 조직 구성 163. 데이터 품질 계획 수립 18【평가영역】구축단계 품질관리,0.067468
2,19,30,"품질관리 담당부서(담당자)가 정의된 역할과 책임에 따른 품질관리 활동을 준수하여야 한다. 의사결정권자(기관장, 책임관 등)에게 데이터 품질 관련 주요사항을 정기적으로 보고하여야 한다.1등급  2등급을 충족하여야 한다.  조직 전체의 데이터 품질관리를 위한 의사결정권자(기관장, 책임관 등)가 직접 참여하는 의사소통 체계(품질관리 정책협의회 등)를 구성하고, 정기적인 의사소통 활동을 실시하여야 한다. (당해년도 포함 최근 2년간, 반기 1회 이상)Ⅱ. 공공데이터 품질관리 수준 평가 지표【평가영역】계획단계 품질관리3. 데이터 품질 계획 수립",0.068327
3,19,26,"- 품질관리 정책, 관련법령, 제도 등의 변경 등에 따른 데이터 품질관리 규정(또는 지침)을 검토한 내역이나 개정이력이 존재하여야 한다.Ⅱ. 공공데이터 품질관리 수준 평가 지표【평가영역】계획단계 품질관리2. 데이터 품질 조직 구성 평가영역 계획단계 품질관리 평가프로세스 2. 데이터 품질 조직 구성 평가지표 2.1 데이터 품질관리 조직 구성 및 의사소통 활동 지표설명  데이터 품질관리 조직을 구성하고, 데이터 품질관리를 위한 의사소통 활동을 수행한다.",0.069262
4,19,28,"증빙자료 ✓ 공공데이터제공책임관 및 실무담당자 임명(변경) 및 홈페이지 공표 내역 ✓ 품질관리 업무별 담당자 지정 및 역할과 책임을 명시한 관련 문서(업무분장표 등)✓ 데이터 품질관리 관련 의사소통 수행내역, 관련 회의록 등평가기준 4등급  공공데이터법 제12조에 근거하여 품질관리 업무를 총괄하는 '공공데이터제공책임관' 및 '실무자'를 임명하여 기관 홈페이지에 공표하여야 한다. - 공공데이터제공책임관은 공공데이터법 시행령 제13조의 임명요건을 충족하여야 한다. 데이터 품질관리를 위한 의사소통 활동이 존재하여야 한다.",0.072597


In [ ]:
hwp to 임베딩 쿼리 시나리오 였슴니다. RAG 체험을 원한다면 다음을 진행하세요